In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (XLMRobertaTokenizerFast, XLMRobertaForTokenClassification, 
                          DistilBertTokenizerFast, DistilBertForTokenClassification,
                          BertTokenizerFast, BertForTokenClassification,
                          Trainer, TrainingArguments, DataCollatorForTokenClassification)
import evaluate

# Load the labeled dataset in CoNLL format
def load_conll_data(file_path):
    """Loads CoNLL formatted data into a pandas DataFrame."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        sentence = []
        labels = []
        for line in f:
            if line.strip():
                token, label = line.strip().split()
                sentence.append(token)
                labels.append(label)
            else:
                data.append((sentence, labels))
                sentence = []
                labels = []
    if sentence:  # For the last sentence if there is no newline
        data.append((sentence, labels))
    return data

conll_file_path = '../output/labeled_telegram_data.conll'
data = load_conll_data(conll_file_path)

# Convert data into a DataFrame
df = pd.DataFrame(data, columns=['tokens', 'labels'])

# Check the size of the dataset
if len(df) > 1:
    # Split into train and validation sets if you have more than one sample
    train_df, val_df = train_test_split(df, test_size=0.2)
else:
    # If the dataset is too small, use the entire dataset for both training and validation
    print("Dataset too small to split. Using the entire dataset for training and evaluation.")
    train_df = df
    val_df = df

# Convert to Hugging Face dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenization and alignment of labels (same function as before)
def tokenize_and_align_labels(examples, tokenizer):
    """Tokenizes inputs and aligns labels."""
    tokenized_inputs = tokenizer(examples['tokens'], is_split_into_words=True, padding=True, truncation=True)

    labels = []
    for i, label in enumerate(examples['labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # get word ids
        label_ids = [-100] * len(tokenized_inputs['input_ids'][i])  # default to -100 (ignore index)

        # Align labels with tokenized inputs
        for j, label_id in enumerate(label):
            if j < len(word_ids) and word_ids[j] is not None:  # avoid IndexError
                if label_id in label_map:  # Check if label_id exists in label_map
                    label_ids[word_ids[j]] = label_map[label_id]  # map label to its id

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

# Map label names to IDs
label_list = list(set(label for labels in df['labels'] for label in labels))
label_list = sorted(label_list)
label_map = {label: i for i, label in enumerate(label_list)}

# Metrics function using Hugging Face's `evaluate` library
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    # Remove ignored index (-100)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Define model names for comparison
model_names = [
    "xlm-roberta-base",  # XLM-Roberta
    "distilroberta-base",  # DistilRoBERTa
    "bert-base-multilingual-cased",  # mBERT
]

# Store results
results = {}

for model_name in model_names:
    print(f"Training model: {model_name}")
    
    # Load tokenizer and model based on the model name
    try:
        if model_name == "xlm-roberta-base":
            tokenizer = XLMRobertaTokenizerFast.from_pretrained(model_name)
            model = XLMRobertaForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))
        elif model_name == "distilroberta-base":
            tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
            model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))
        elif model_name == "bert-base-multilingual-cased":
            tokenizer = BertTokenizerFast.from_pretrained(model_name)
            model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

        # Initialize the data collator (moved inside the loop)
        data_collator = DataCollatorForTokenClassification(tokenizer, padding=True)

    except Exception as e:
        print(f"Error loading model {model_name}: {e}")
        continue

    # Tokenize the datasets
    tokenized_train_dataset = train_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer), 
                                                batched=True, remove_columns=['tokens', 'labels'])
    tokenized_val_dataset = val_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer), 
                                             batched=True, remove_columns=['tokens', 'labels'])

    # Set up training arguments
    training_args = TrainingArguments(
        output_dir=f'../results/{model_name}',  # output directory
        evaluation_strategy="epoch",           # evaluate every epoch
        learning_rate=2e-5,                    # learning rate
        per_device_train_batch_size=16,        # batch size for training
        per_device_eval_batch_size=16,         # batch size for evaluation
        num_train_epochs=3,                    # total number of training epochs
        weight_decay=0.01,                     # strength of weight decay
        logging_dir=f'../logs/{model_name}',   # directory for storing logs
    )

    # Create a Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset,   # Use validation set for evaluation
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Start training
    trainer.train()

    # Evaluate the fine-tuned model on the validation set
    eval_results = trainer.evaluate()
    results[model_name] = eval_results
    print(f"Evaluation results for {model_name}: {eval_results}")

# Print summary of results
print("\nModel Comparison Results:")
for model_name, result in results.items():
    print(f"{model_name}: Precision={result['eval_precision']:.4f}, Recall={result['eval_recall']:.4f}, F1={result['eval_f1']:.4f}, Accuracy={result['eval_accuracy']:.4f}")

# Step 7: Save the fine-tuned models
for model_name in model_names:
    model.save_pretrained(f'../models/{model_name}')
    tokenizer.save_pretrained(f'../models/{model_name}')
    print(f"Model and tokenizer for {model_name} saved successfully!")
